In [1]:
import tkinter as tk
from tkinter import messagebox
from sqlalchemy import create_engine, text

# Configuração de conexão com o banco de dados
DB_USER = 'postgres'
DB_PASSWORD = '1010'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'DB_Final'

def connect_db():
    try:
        conexao_bd = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
        engine = create_engine(
            conexao_bd,
            connect_args={'options': '-csearch_path=sistema,public'}
        )
        conn = engine.connect()
        return conn
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return None

def insert_secretario(nome, cpf, emails, telefones, rua, numero, bairro, cidade, estado, horarios, responsabilidades):
    conn = connect_db()  # Mantendo a função de conexão com o banco
    if conn:
        try:
            # Início da transação
            trans = conn.begin()

            # Inserir na tabela Pessoa
            pessoa_query = text("INSERT INTO Pessoa (Nome, CPF) VALUES (:nome, :cpf) RETURNING ID_Pessoa")
            pessoa_result = conn.execute(pessoa_query, {"nome": nome, "cpf": cpf})
            id_pessoa = pessoa_result.fetchone()[0]
            print(f"ID_Pessoa: {id_pessoa}")
            
            # Inserir múltiplos emails
            email_query = text("INSERT INTO Email (Email, ID_Pessoa) VALUES (:email, :id_pessoa)")
            for email in emails.split(','):
                conn.execute(email_query, {"email": email.strip(), "id_pessoa": id_pessoa})
            
            # Inserir múltiplos telefones
            telefone_query = text("INSERT INTO Telefone (Telefone, ID_Pessoa) VALUES (:telefone, :id_pessoa)")
            for telefone in telefones.split(','):
                conn.execute(telefone_query, {"telefone": telefone.strip(), "id_pessoa": id_pessoa})
            
            # Inserir na tabela Endereco
            endereco_query = text("INSERT INTO Endereco (Rua, Numero, Bairro, Cidade, Estado, ID_Pessoa) VALUES (:rua, :numero, :bairro, :cidade, :estado, :id_pessoa)")
            conn.execute(endereco_query, {"rua": rua, "numero": numero, "bairro": bairro, "cidade": cidade, "estado": estado, "id_pessoa": id_pessoa})
            
            # Inserir na tabela Secretario
            secretario_query = text("INSERT INTO Secretario (ID_Pessoa) VALUES (:id_pessoa) RETURNING ID_Secretario")
            secretario_result = conn.execute(secretario_query, {"id_pessoa": id_pessoa})
            id_secretario = secretario_result.fetchone()[0]
            print(f"ID_Secretario: {id_secretario}")
            
            # Inserir responsabilidades do secretário
            responsabilidade_query = text("INSERT INTO Responsabilidade (Descricao, ID_Secretario) VALUES (:descricao, :id_secretario)")
            for responsabilidade in responsabilidades.split(','):
                conn.execute(responsabilidade_query, {"descricao": responsabilidade.strip(), "id_secretario": id_secretario})
            
            # Inserir horários de atendimento na tabela Horario_Atendimento_Secretario
            horario_query = text("""
            INSERT INTO Horario_Atendimento_Secretario 
            (ID_Dia_Semana, Hora_Inicio_Manha, Hora_Fim_Manha, Hora_Inicio_Tarde, Hora_Fim_Tarde, Hora_Inicio_Noite, Hora_Fim_Noite, ID_Secretario) 
            VALUES (:dia_semana, :hora_inicio_manha, :hora_fim_manha, :hora_inicio_tarde, :hora_fim_tarde, :hora_inicio_noite, :hora_fim_noite, :id_secretario)
            """)
            for horario in horarios:
                horario_parametros = {
                    "dia_semana": horario['dia_semana'],
                    "hora_inicio_manha": horario['hora_inicio_manha'],
                    "hora_fim_manha": horario['hora_fim_manha'],
                    "hora_inicio_tarde": horario['hora_inicio_tarde'],
                    "hora_fim_tarde": horario['hora_fim_tarde'],
                    "hora_inicio_noite": horario['hora_inicio_noite'],
                    "hora_fim_noite": horario['hora_fim_noite'],
                    "id_secretario": id_secretario
                }
                print(f"Inserindo Horário: {horario_parametros}")
                conn.execute(horario_query, horario_parametros)

            # Commit da transação
            trans.commit()
            messagebox.showinfo("Sucesso", "Secretário cadastrado com sucesso!")
        except Exception as e:
            trans.rollback()
            messagebox.showerror("Erro", f"Ocorreu um erro ao cadastrar o secretário: {e}")
            print(f"Erro ao cadastrar secretário: {str(e)}")
        finally:
            conn.close()

# Função para capturar os dados da interface e chamar a função de inserção
def salvar_secretario():
    nome = entry_nome.get()
    cpf = entry_cpf.get()
    emails = entry_email.get()  # Suporta múltiplos emails separados por vírgula
    telefones = entry_telefone.get()  # Suporta múltiplos telefones separados por vírgula
    rua = entry_rua.get()
    numero = entry_numero.get()
    bairro = entry_bairro.get()
    cidade = entry_cidade.get()
    estado = entry_estado.get()
    responsabilidades = entry_responsabilidades.get()  # Responsabilidades do secretário

    # Verifica quais dias da semana estão selecionados
    dias_semana_selecionados = []
    if var_domingo.get(): dias_semana_selecionados.append(1)
    if var_segunda.get(): dias_semana_selecionados.append(2)
    if var_terca.get(): dias_semana_selecionados.append(3)
    if var_quarta.get(): dias_semana_selecionados.append(4)
    if var_quinta.get(): dias_semana_selecionados.append(5)
    if var_sexta.get(): dias_semana_selecionados.append(6)
    if var_sabado.get(): dias_semana_selecionados.append(7)

    # Captura os horários preenchidos para cada dia selecionado
    horarios = []
    for dia in dias_semana_selecionados:
        horarios.append({
            'dia_semana': dia,
            'hora_inicio_manha': entry_hora_inicio_manha.get() or None,
            'hora_fim_manha': entry_hora_fim_manha.get() or None,
            'hora_inicio_tarde': entry_hora_inicio_tarde.get() or None,
            'hora_fim_tarde': entry_hora_fim_tarde.get() or None,
            'hora_inicio_noite': entry_hora_inicio_noite.get() or None,
            'hora_fim_noite': entry_hora_fim_noite.get() or None,
        })
    
    # Inserir os dados no banco
    insert_secretario(nome, cpf, emails, telefones, rua, numero, bairro, cidade, estado, horarios, responsabilidades)

# Interface gráfica usando Tkinter
root = tk.Tk()
root.title("Cadastro de Secretário")

# Labels e campos de texto para as informações do secretário
tk.Label(root, text="Nome").grid(row=0, column=0)
entry_nome = tk.Entry(root)
entry_nome.grid(row=0, column=1)

tk.Label(root, text="CPF").grid(row=1, column=0)
entry_cpf = tk.Entry(root)
entry_cpf.grid(row=1, column=1)

tk.Label(root, text="Email").grid(row=2, column=0)
entry_email = tk.Entry(root)
entry_email.grid(row=2, column=1)

tk.Label(root, text="Telefone").grid(row=3, column=0)
entry_telefone = tk.Entry(root)
entry_telefone.grid(row=3, column=1)

tk.Label(root, text="Rua").grid(row=4, column=0)
entry_rua = tk.Entry(root)
entry_rua.grid(row=4, column=1)

tk.Label(root, text="Número").grid(row=5, column=0)
entry_numero = tk.Entry(root)
entry_numero.grid(row=5, column=1)

tk.Label(root, text="Bairro").grid(row=6, column=0)
entry_bairro = tk.Entry(root)
entry_bairro.grid(row=6, column=1)

tk.Label(root, text="Cidade").grid(row=7, column=0)
entry_cidade = tk.Entry(root)
entry_cidade.grid(row=7, column=1)

tk.Label(root, text="Estado").grid(row=8, column=0)
entry_estado = tk.Entry(root)
entry_estado.grid(row=8, column=1)

# Campo para responsabilidades
tk.Label(root, text="Responsabilidades").grid(row=9, column=0)
entry_responsabilidades = tk.Entry(root)
entry_responsabilidades.grid(row=9, column=1)

# Criação dos checkboxes para os dias da semana
tk.Label(root, text="Dias da Semana:").grid(row=10, column=0)

var_domingo = tk.IntVar()
var_segunda = tk.IntVar()
var_terca = tk.IntVar()
var_quarta = tk.IntVar()
var_quinta = tk.IntVar()
var_sexta = tk.IntVar()
var_sabado = tk.IntVar()

tk.Checkbutton(root, text="Domingo", variable=var_domingo).grid(row=11, column=0, sticky='w')
tk.Checkbutton(root, text="Segunda-feira", variable=var_segunda).grid(row=11, column=1, sticky='w')
tk.Checkbutton(root, text="Terça-feira", variable=var_terca).grid(row=11, column=2, sticky='w')
tk.Checkbutton(root, text="Quarta-feira", variable=var_quarta).grid(row=12, column=0, sticky='w')
tk.Checkbutton(root, text="Quinta-feira", variable=var_quinta).grid(row=12, column=1, sticky='w')
tk.Checkbutton(root, text="Sexta-feira", variable=var_sexta).grid(row=12, column=2, sticky='w')
tk.Checkbutton(root, text="Sábado", variable=var_sabado).grid(row=13, column=0, sticky='w')

# Campos para horários
tk.Label(root, text="Hora Início Manhã").grid(row=14, column=0)
entry_hora_inicio_manha = tk.Entry(root)
entry_hora_inicio_manha.grid(row=14, column=1)

tk.Label(root, text="Hora Fim Manhã").grid(row=15, column=0)
entry_hora_fim_manha = tk.Entry(root)
entry_hora_fim_manha.grid(row=15, column=1)

tk.Label(root, text="Hora Início Tarde").grid(row=16, column=0)
entry_hora_inicio_tarde = tk.Entry(root)
entry_hora_inicio_tarde.grid(row=16, column=1)

tk.Label(root, text="Hora Fim Tarde").grid(row=17, column=0)
entry_hora_fim_tarde = tk.Entry(root)
entry_hora_fim_tarde.grid(row=17, column=1)

tk.Label(root, text="Hora Início Noite").grid(row=18, column=0)
entry_hora_inicio_noite = tk.Entry(root)
entry_hora_inicio_noite.grid(row=18, column=1)

tk.Label(root, text="Hora Fim Noite").grid(row=19, column=0)
entry_hora_fim_noite = tk.Entry(root)
entry_hora_fim_noite.grid(row=19, column=1)

# Botão de salvar
btn_salvar = tk.Button(root, text="Salvar", command=salvar_secretario)
btn_salvar.grid(row=20, column=1)

# Loop da interface gráfica
root.mainloop()


ID_Pessoa: 24
ID_Secretario: 4
Inserindo Horário: {'dia_semana': 2, 'hora_inicio_manha': '08:00', 'hora_fim_manha': '11:00', 'hora_inicio_tarde': '13:30', 'hora_fim_tarde': '17:00', 'hora_inicio_noite': None, 'hora_fim_noite': None, 'id_secretario': 4}
Inserindo Horário: {'dia_semana': 3, 'hora_inicio_manha': '08:00', 'hora_fim_manha': '11:00', 'hora_inicio_tarde': '13:30', 'hora_fim_tarde': '17:00', 'hora_inicio_noite': None, 'hora_fim_noite': None, 'id_secretario': 4}
Inserindo Horário: {'dia_semana': 4, 'hora_inicio_manha': '08:00', 'hora_fim_manha': '11:00', 'hora_inicio_tarde': '13:30', 'hora_fim_tarde': '17:00', 'hora_inicio_noite': None, 'hora_fim_noite': None, 'id_secretario': 4}
Inserindo Horário: {'dia_semana': 5, 'hora_inicio_manha': '08:00', 'hora_fim_manha': '11:00', 'hora_inicio_tarde': '13:30', 'hora_fim_tarde': '17:00', 'hora_inicio_noite': None, 'hora_fim_noite': None, 'id_secretario': 4}
Inserindo Horário: {'dia_semana': 6, 'hora_inicio_manha': '08:00', 'hora_fim_manh